## Installations

In [1]:
%pip install -q torch transformers accelerate sentence-transformers faiss-gpu langchain langchain_community
%pip install -q httpx==0.27.2 openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.3 MB/s eta 0:00:00


In [2]:
!gdown 1FLg1UlQuH6E7fzIFeiATsawIby48hQs3

Downloading...
From: https://drive.google.com/uc?id=1FLg1UlQuH6E7fzIFeiATsawIby48hQs3
To: /content/rag-assistant.zip
100% 1.88M/1.88M [00:00<00:00, 57.1MB/s]


In [3]:
!unzip rag-assistant.zip

Archive:  rag-assistant.zip
  inflating: embeddings_openai.pickle  
  inflating: __MACOSX/._embeddings_openai.pickle  
  inflating: questions.pickle        
  inflating: __MACOSX/._questions.pickle  
  inflating: modified_questions.json  
  inflating: __MACOSX/._modified_questions.json  


## imports and settings

In [4]:
import pickle

import json
import numpy as np
from tqdm import tqdm

import tiktoken
from openai import OpenAI

import faiss
from langchain.embeddings import HuggingFaceEmbeddings

from google.colab import userdata

In [5]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [6]:
with open('questions.pickle', 'rb') as handle:
    questions = pickle.load(handle)

questions = np.array(questions)

In [7]:
def get_map5_score(questions, get_context_elements):
  map5 = []

  with open(questions, 'r', encoding='utf-8') as f:
    dict_with_questions = json.load(f)

  for item in tqdm(dict_with_questions['questions']):
    original_question = item['original']
    modified_question = item['modified']
    result_of_retrieving = [str(x) for x in get_context_elements(modified_question)]

    for i, element in enumerate(result_of_retrieving):
      if original_question in element:
        map5.append((5 - i) / len(result_of_retrieving))
        break
    else:
      map5.append(0)

  return np.mean(map5), np.std(map5 , ddof=1)

In [8]:
def get_acc5_score(questions, get_context_elements):
  acc5 = []

  with open(questions, 'r', encoding='utf-8') as f:
    dict_with_questions = json.load(f)

  for item in tqdm(dict_with_questions['questions']):
    original_question = item['original']
    modified_question = item['modified']
    result_of_retrieving = [str(x) for x in get_context_elements(modified_question)]

    for element in result_of_retrieving:
      if original_question in element:
        acc5.append(1)
        break
    else:
        acc5.append(0)

  return np.mean(acc5), np.std(acc5, ddof=1)

## OpenAI

In [23]:
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def price_of_embedding_request(string: str) -> float:
  # https://openai.com/api/pricing/#:~:text=text%2Dembedding%2D3,0.020%20/%201M%20tokens
  price_per_token = 0.02 / 1_000_000
  tokens = num_tokens_from_string(string)
  return tokens * price_per_token

In [24]:
total_price = 0

for question in tqdm(questions):
  total_price += price_of_embedding_request(question)

print()
print(f"${total_price}" )

100%|██████████| 299/299 [00:01<00:00, 295.25it/s]


$0.0016315200000000002


In [25]:
client = OpenAI(api_key=OPENAI_API_KEY)

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input=[text], model=model).data[0].embedding

In [ ]:
## embeddings_openai = []

## for question in tqdm(questions):
##   embedding = get_embedding(question)
##   embeddings_openai.append(embedding)

In [27]:
with open('embeddings_openai.pickle', 'rb') as handle:
    embeddings_openai = pickle.load(handle)

embeddings_openai = np.float32(embeddings_openai) # equivalent to previous cell

In [28]:
print(embeddings_openai.shape[1])
db = faiss.IndexFlatL2(embeddings_openai.shape[1])
db.add(embeddings_openai)

1536


In [29]:
def get_context_elements_openai(query):
  query_embedding = np.float32([get_embedding(query)])
  # Perform the search
  D, I = db.search(query_embedding, k=5)  # k is the number of nearest neighbors
  return questions[I][0]

In [30]:
query = "Який тариф обрати, якщо я буду використовувати лише дзвінки?"
print(get_context_elements_openai(query))

['1\\. **Які є тарифи тільки для дзвінків?**   \nПропонуємо подивитися умови тарифу LOVE UA На максимум з підключеною Суперсилою Економія. У вас буде безлім на Київстар, 500 МБ Інтернету та ще одна Суперсилу на вибір. Вартість тарифу — 125 грн/4 тижні (заощаджуйте 50 грн, від звичайної вартості 175 грн/4 тижні).'
 '9\\. **Як дізнатися свій тарифний план?** Є декілька способів:\n\n* за запитом \\*100\\*01\\*2\\#  \n* авторизуйтеся на сайті та перейдіть на сторінку Тарифи  \n* у додатку Мій Київстар  \n* за номером 466\\*66'
 '6\\. **Як я можу заощадити завдяки тарифам Все разом?** Наприклад, ви вирішили підключити мобільний номер, вибравши тариф LOVE UA Сила (330 грн/4 тижні), Домашній Інтернет на швидкості 100 Мбіт/с (330 грн/міс) і Київстар ТБ Преміум HD (200 грн/міс) — разом 860 грн/міс. За той же обсяг послуг у тарифі ВСЕ РАЗОМ Міць ви заплатите всього 500 грн/міс, отже, щомісяця вигода становитиме 360 грн. Також набагато зручніше сплачувати за всі послуги один раз на місяць, поповн

In [33]:
get_map5_score('modified_questions.json', get_context_elements_openai)

100%|██████████| 51/51 [00:14<00:00,  3.56it/s]


(0.7725490196078431, 0.3688243112228634)

In [34]:
get_acc5_score('modified_questions.json', get_context_elements_openai)

100%|██████████| 51/51 [00:09<00:00,  5.38it/s]


(0.8627450980392157, 0.34754037711536506)

## intfloat/multilingual-e5-large


In [9]:
embd_model_e5 = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

<ipython-input-9-4ccb5cf72286>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embd_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [16]:
embeddings_e5 = []

for question in tqdm(questions):
  embeddings_e5.append( embd_model_e5.embed_query(question) )

embeddings_e5 = np.float32(embeddings_e5)

100%|██████████| 299/299 [00:11<00:00, 26.87it/s]


In [18]:
print(embeddings_e5.shape[1])
db = faiss.IndexFlatL2(embeddings_e5.shape[1])
db.add(embeddings_e5)

1024


In [19]:
def get_context_elements_e5(query):
  query_embedding = np.float32([embd_model_e5.embed_query(query)])
  # Perform the search
  D, I = db.search(query_embedding, k=5)  # k is the number of nearest neighbors
  return questions[I][0]

In [20]:
query = "Який тариф обрати, якщо я буду використовувати лише дзвінки?"
print(get_context_elements_e5(query))

['1\\. **Які є тарифи тільки для дзвінків?**   \nПропонуємо подивитися умови тарифу LOVE UA На максимум з підключеною Суперсилою Економія. У вас буде безлім на Київстар, 500 МБ Інтернету та ще одна Суперсилу на вибір. Вартість тарифу — 125 грн/4 тижні (заощаджуйте 50 грн, від звичайної вартості 175 грн/4 тижні).'
 '4. **Як заборонити/дозволити вхідні або вихідні дзвінки та SMS?**  \n   Заборонити вхідні дзвінки та SMS можуть тільки контрактні абоненти за допомогою послуги  \n   «Заборона виклику». Додаткова абонентська плата не стягується.  \n   Управляти послугою можна за допомогою функцій мобільного телефону та спеціальних службових команд.  \n   Деталі – в описі послуги «Заборона виклику».'
 "24\\. **Як залишатися на зв'язку при перевищенні кредиту або нульовому балансі?** Якщо вам потрібно подзвонити, скористайтеся послугами Короткий виклик – і ваш співрозмовник оплатить розмову."
 '9. **Як змінити тариф?**  \n   На головному екрані додатка натисніть на кнопку «Змінити тариф». Обер

In [21]:
get_map5_score('modified_questions.json', get_context_elements_e5)

100%|██████████| 51/51 [00:01<00:00, 41.64it/s]


(0.8235294117647058, 0.3314140825578284)

In [22]:
get_acc5_score('modified_questions.json', get_context_elements_e5)

100%|██████████| 51/51 [00:01<00:00, 33.42it/s]


(0.8823529411764706, 0.3253956867279843)

## sentence-transformers/distiluse-base-multilingual-cased-v2


In [35]:
embd_model_distiluse = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [36]:
embeddings_distiluse = []

for question in tqdm(questions):
  embeddings_distiluse.append( embd_model_distiluse.embed_query(question) )

embeddings_distiluse = np.float32(embeddings_distiluse)

100%|██████████| 299/299 [00:02<00:00, 142.31it/s]


In [37]:
print(embeddings_distiluse.shape[1])
db = faiss.IndexFlatL2(embeddings_distiluse.shape[1])
db.add(embeddings_distiluse)

512


In [38]:
def get_context_elements_distiluse(query):
  query_embedding = np.float32([embd_model_distiluse.embed_query(query)])
  # Perform the search
  D, I = db.search(query_embedding, k=5)  # k is the number of nearest neighbors
  return questions[I][0]

In [39]:
query = "Який тариф обрати, якщо я буду використовувати лише дзвінки?"
print(get_context_elements_distiluse(query))

['27\\. **Як попросити знайомого поповнити мій рахунок, якщо я не маю грошей на дзвінок чи повідомлення?** Скористайтеся безкоштовною послугою «Поповни мій рахунок». Просто наберіть наступну команду: \\*132\\*380YYХХХХХХХ\\#, де YY — код мережі Київстар, а ХХХХХХХ — номер телефону абонента, якому ви бажаєте надіслати запит. Протягом декількох секунд цей абонент отримає SMS-повідомлення із проханням поповнити рахунок. При цьому на вашому рахунку може бути будь-яка сума. Деталі – в описі послуги Поповни мій рахунок.'
 '9\\. **Чи можу я використати залишок на рахунку Домашнього Інтернету, щоб оплатити тариф Все разом або перенести кошти на мобільний номер?** У тарифах Все разом плата за всі послуги знімається тільки з мобільного рахунку. Перенести гроші з рахунку послуги Домашній Інтернет на ваш мобільний номер неможливо. Однак гроші нікуди не зникають, а зберігатимуться на вашому особовому рахунку. Їх можна буде використати, наприклад, якщо ви захочете підключити окремий тариф для Домашн

In [40]:
get_map5_score('modified_questions.json', get_context_elements_distiluse)

100%|██████████| 51/51 [00:00<00:00, 158.07it/s]


(0.5333333333333334, 0.4554850894010326)

In [41]:
get_acc5_score('modified_questions.json', get_context_elements_distiluse)

100%|██████████| 51/51 [00:00<00:00, 76.32it/s]


(0.6274509803921569, 0.48829435031445906)